# 1. Imports & Environment Setup

- **Core Imports:**
  - `os`, `warnings`, `datetime`, `uuid`, `string`, `random`, `hashlib`, `traceback`.

- **LangChain/LangGraph Imports:**
  - `ChatOpenAI`, `StreamingStdOutCallbackHandler`, `TavilySearch`, `TavilySearchResults`, agent/graph/integration tools, message types.

- **Notebook & Display:**
  - `clear_output`, `Markdown` from `IPython.display`.

- **Environment:**
  - Suppresses TensorFlow warnings.
  - Loads environment variables (e.g., API keys) using `python-dotenv`.


In [1]:
import os
import warnings
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_tavily import TavilySearch
from langchain_community.tools import TavilySearchResults
from langchain.agents import create_tool_calling_agent, AgentExecutor
from typing import TypedDict, Dict, List
from langchain_core.messages import SystemMessage, AIMessage, HumanMessage
from datetime import datetime
from IPython.display import clear_output, Markdown
import traceback
import uuid
import string
import random
import hashlib

# 2. Key Utility Functions

get_api_key(key_name="OPENROUTER_API_KEY"):
Retrieves an API key from environment variables, raises an error if not found.

initialize_llm(...):
Initializes the language model (LLM) with streaming and callback support.

save_file(data, filename, uniqueness_method="uuid"):
Saves output (e.g., generated plans) to a Markdown file with various unique-naming strategies.

show_md_file(file_path):
Displays a Markdown file’s content within the notebook.

In [2]:
# Suppress TensorFlow and CUDA warnings for a cleaner notebook output
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
warnings.filterwarnings("ignore")

# Load all environment variables (such as API keys) from .env file
load_dotenv()

def get_api_key(key_name="OPENROUTER_API_KEY"):
    """
    Retrieve an API key from environment variables.

    Args:
        key_name (str): Name of the environment variable.

    Returns:
        str: Retrieved API key.

    Raises:
        ValueError: If no key is found.
    """
    api_key = os.getenv(key_name)
    if not api_key:
        raise ValueError(f"Invalid API key: {key_name} not found in environment variables")
    return api_key

def initialize_llm(model_name="meta-llama/llama-3.1-8b-instruct", temperature=0.4, use_streaming=True):
    """
    Initialize a streaming Large Language Model (LLM) with specified parameters.

    Args:
        model_name (str): The model's name.
        temperature (float): Sampling temperature for generation.
        use_streaming (bool): Whether to enable streaming output.

    Returns:
        ChatOpenAI: Instance of the language model.
    """
    api_key = get_api_key()
    callbacks = [StreamingStdOutCallbackHandler()]
    llm = ChatOpenAI(
        model_name=model_name,
        temperature=temperature,
        streaming=use_streaming,
        callbacks=callbacks,
        openai_api_key=api_key,
        openai_api_base="https://openrouter.ai/api/v1"
    )
    return llm

# Initialize the language model for use throughout the notebook
llm = initialize_llm()

python-dotenv could not parse statement starting at line 12


In [3]:
def save_file(data, filename, uniqueness_method="uuid"):
    """
    Save data as a markdown file in a uniquely named manner.

    Args:
        data (str): Content to save.
        filename (str): Base filename.
        uniqueness_method (str): How to make filenames unique (uuid, random_string, hash, counter, datetime).

    Returns:
        str: Path to the saved file.
    """
    folder_name = "Travel Plans"
    os.makedirs(folder_name, exist_ok=True)  # Ensure folder exists

    # Choose unique identifier method for filename
    if uniqueness_method == "uuid":
        unique_id = str(uuid.uuid4())[:8]
    elif uniqueness_method == "random_string":
        chars = string.ascii_letters + string.digits
        unique_id = ''.join(random.choice(chars) for _ in range(8))
    elif uniqueness_method == "hash":
        content_hash = hashlib.md5((data + str(datetime.now())).encode()).hexdigest()[:8]
        unique_id = content_hash
    elif uniqueness_method == "counter":
        counter = 1
        while True:
            test_filename = f"{filename}_{counter}.md"
            test_path = os.path.join(folder_name, test_filename)
            if not os.path.exists(test_path):
                unique_id = str(counter)
                break
            counter += 1
    elif uniqueness_method == "datetime":
        unique_id = datetime.now().strftime("%Y%m%d%H%M%S")
    else:
        unique_id = str(uuid.uuid4())[:8]

    final_filename = f"{filename}_{unique_id}.md"
    file_path = os.path.join(folder_name, final_filename)

    # Write data to file
    with open(file_path, "w", encoding="utf-8") as file:
        file.write(data)
        print(f"File '{file_path}' created successfully.")

    return file_path

def show_md_file(file_path):
    """
    Display the content of a markdown file within the notebook.

    Args:
        file_path (str): Path to the markdown file.
    """
    with open(file_path, "r", encoding = "utf-8") as file:
        content = file.read()
    display(Markdown(content))

# 3. Planner State Structure
Defines a TypedDict, PlannerState, which holds all state for the travel planning workflow:

- travel_preferences: User’s trip preferences.
- destination_options: Suggested destinations.
- selected_destination: User’s chosen destination.
- budget_plan: Trip budget breakdown.
- itinerary: Day-by-day trip plan.
- bookings: Booking recommendations.
- feedback: User feedback.
- final_trip: The final trip package.
messages: Message history (for context).
status: Workflow status string.

In [4]:
class PlannerState(TypedDict):
    """
    Structure representing the entire travel planning state across all workflow nodes.
    """
    travel_preferences: Dict
    destination_options: List[Dict]
    selected_destination: Dict
    budget_plan: Dict
    itinerary: List[Dict]
    bookings: Dict
    feedback: Dict
    final_trip: Dict
    messages: List
    status: str

# 4. Workflow Node Functions
Each node is a function that takes and returns a PlannerState object, incrementally enriching the plan:

- input_collector:
    - Extracts user travel preferences from the message history via the LLM.
- destination_research: 
    - Suggests 3–5 destinations matching the user’s preferences, with descriptions, costs, and timing.
- collect_selected_destination:
    - Captures which destination the user selects from the options.
- budget_planner:
    - Breaks down the trip’s expected costs (transport, accommodation, activities, etc.).
- itinerary_builder:
    - Generates a detailed multi-day plan, balancing activity and rest.
- booking_assistant: 
    - Provides booking recommendations (flights, hotels, activities, insurance, etc.), with tips and links.
- trip_exporter: 
    - Compiles the entire trip—including preferences, itinerary, budget, bookings, packing tips, and emergency contacts—into a final Markdown document.



In [5]:
def input_collector(state: PlannerState) -> PlannerState:
    """
    Collects and organizes user's travel preferences from messages.

    Args:
        state (PlannerState): The current workflow state.

    Returns:
        PlannerState: Updated state with extracted preferences.
    """
    messages = state["messages"]
    # Extract the latest user message
    human_message = next((m for m in reversed(messages) if isinstance(m, HumanMessage)), None)
    system_prompt = (
        "You are a travel input collector. Extract travel preferences from the user's message.\n"
        "Include: destination interests, travel dates, budget range, number of travelers, "
        "accommodation preferences, activity interests, and any special requirements."
    )
    response = llm.invoke([
        SystemMessage(content=system_prompt),
        human_message
    ])
    preferences = {
        "raw_input": human_message.content,
        "processed_preferences": response.content,
        "timestamp": datetime.now().isoformat()
    }
    
    #Update state
    state["travel_preferences"] = preferences
    state["messages"].append(AIMessage(content="I've collected your travel preferences. Moving on to destination research."))
    state["status"] = "preferences_collected"
    return state

In [6]:
def destination_research(state: PlannerState) -> PlannerState:
    """
    Research and suggest possible destinations based on user preferences.

    Args:
        state (PlannerState): The current workflow state.

    Returns:
        PlannerState: Updated state with destination suggestions.
    """
    preferences = state["travel_preferences"]
    system_prompt = """
        You are a destination research expert with extensive global travel knowledge.

        Based on the user's comprehensive travel preferences, suggest 3–5 meticulously selected destinations.

        For each destination, provide:

        1. **Name & Description**:
        - Detailed name
        - Compelling description highlighting unique features

        2. **Preference Alignment**:
        - Explanation of why it matches their specific preferences

        3. **Seasonal Considerations**:
        - Ideal visiting months
        - Weather/climate context

        4. **Cost Analysis**:
        - Luxury, mid-range, and budget cost breakdowns

        5. **Attractions** (categorized):
        - Cultural
        - Natural
        - Adventure
        - Other relevant categories

        6. **Local Transportation**:
        - Public and private options

        7. **Safety Considerations**:
        - Travel advisories
        - Health/safety tips

        8. **Visa Requirements**:
        - Entry rules by nationality (generalized)

        9. **Language Considerations**:
        - Common languages
        - Tips for non-native speakers

        10. **Insider Recommendations**:
            - Hidden gems
            - Local customs
            - Experiences enhancing their travel style
            - Provide links to websites if possible

        Also include **at least one unexpected but highly relevant destination** to broaden their perspective.
        """

    response = llm.invoke([
        SystemMessage(content=system_prompt),
        HumanMessage(content=f"Travel Preferences: {preferences}")
    ])
    destination_options = {
        "options": response.content,
        "research_timestamp": datetime.now().isoformat()
    }
    state["destination_options"] = destination_options
    state["messages"].append(AIMessage(content="I've researched some destinations that match your preferences. Moving on to budget planning."))
    state["status"] = "awaiting_destination_selection"
    return state

In [7]:
def collect_selected_destination(state: PlannerState) -> PlannerState:
    """
    Capture the user's selected destination from the message history.

    Args:
        state (PlannerState): The current workflow state.

    Returns:
        PlannerState: Updated state with the user's destination choice.
    """
    messages = state["messages"]
    latest_human = next((m for m in reversed(messages) if isinstance(m, HumanMessage)), None)
    if not latest_human:
        raise ValueError("No user input found for destination selection.")

    selected_destination = {
        "name": latest_human.content,
        "details": "To be looked up or parsed from previous suggestions."
    }
    state["selected_destination"] = selected_destination
    state["messages"].append(AIMessage(content=f"Thanks! You've selected **{latest_human.content}**. We'll proceed to budget planning."))
    state["status"] = "destination_selected"
    return state

In [8]:
def budget_planner(state: PlannerState) -> PlannerState:
    """
    Create a detailed budget breakdown for the user's trip.

    Args:
        state (PlannerState): The current workflow state.

    Returns:
        PlannerState: Updated state with the trip budget plan.
    """
    preferences = state["travel_preferences"]
    destination = state["selected_destination"]
    system_prompt = """
        You are a travel budget planner with expertise in global financial planning.

        Create an extensively detailed budget breakdown for the trip, including:

        1. **Transportation Costs**
        - International flights
        - Regional connections
        - Airport transfers
        - Daily local transit options

        2. **Accommodation Costs**
        - Based on selected property types
        - Include seasonal price variations

        3. **Daily Expenses**
        - Breakfast, lunch, dinner, snacks
        - Guided tours and self-guided activities
        - Entrance fees
        - Tipping customs by region

        4. **Entertainment Budget**
        - Events, shows, nightlife, excursions

        5. **Shopping Allowances**
        - Souvenirs and discretionary spending

        6. **Communication Expenses**
        - SIM cards, WiFi, data plans

        7. **Recommended Spending Money**
        - Adjusted for destination-specific cost levels

        8. **Extras & Contingencies**
        - Unexpected costs
        - Emergency buffer (with percentage recommendations)

        9. **Currency Exchange Considerations**
        - Local exchange rates
        - Preferred exchange methods

        10. **Money-Saving Strategies**
            - Discounts, passes, bundled offers
            - Budget travel hacks

        11. **Payment Method Recommendations**
            - Cash vs card vs mobile wallets
            - Region-specific guidance

        Be sure to present this budget in both:
        - **Per-person cost**
        - **Total group cost**
        """

    response = llm.invoke([
        SystemMessage(content=system_prompt),
        HumanMessage(content=f"Travel preferences: {preferences}, Selected destination: {destination}")
    ])
    budget_plan = {
        "breakdown": response.content,
        "timestamp": datetime.now().isoformat()
    }
    state["budget_plan"] = budget_plan
    state["messages"].append(AIMessage(content="I've created a budget plan for your trip. Now building your itinerary."))
    state["status"] = "budget_planned"
    return state

In [9]:
def itinerary_builder(state: PlannerState) -> PlannerState:
    """
    Build a detailed day-by-day itinerary for the trip.

    Args:
        state (PlannerState): The current workflow state.

    Returns:
        PlannerState: Updated state with the trip itinerary.
    """
    preferences = state["travel_preferences"]
    destination = state["selected_destination"]
    budget = state["budget_plan"]
    system_prompt = """
        You are a travel itinerary expert specializing in optimized travel experiences.

        Create a meticulously detailed **day-by-day itinerary** for the trip, including:

        1. **Arrival & Departure Logistics**
        - Strategic planning for flights and check-ins
        - Recovery time considerations after travel

        2. **Daily Activities & Attractions**
        - Thoughtfully sequenced to reduce transit time
        - Maximize cultural and experiential value

        3. **Meal Planning**
        - Restaurant recommendations for each meal
        - Include alternative options for variety and flexibility

        4. **Rest & Recovery**
        - Appropriately timed breaks based on activity level

        5. **Special Experiences**
        - Unique highlights strategically placed across the itinerary

        6. **Contingency Planning**
        - Suggestions for weather disruptions or closures

        7. **Timing & Accessibility**
        - Opening hours of all scheduled attractions
        - Walking distances and expected durations

        8. **Transit Planning**
        - Local transportation options with estimated travel times

        9. **Crowd Avoidance Tips**
        - Location-specific strategies to skip long lines

        10. **Photography & Visual Highlights**
            - Recommended photo opportunities by location and time of day

        11. **Cultural Etiquette Reminders**
            - Important do's and don'ts for each region or activity

        Ensure the itinerary:
        - Is **balanced** to avoid exhaustion
        - **Maximizes authenticity and enjoyment**
        - Respects **budget constraints** and **traveler preferences**
        """
    response = llm.invoke([
        SystemMessage(content=system_prompt),
        HumanMessage(content=f"Travel preferences: {preferences}, Selected destination: {destination}, Budget plan: {budget}")
    ])
    itinerary = {
        "daily_plan": response.content,
        "timestamp": datetime.now().isoformat()
    }
    state["itinerary"] = itinerary
    state["messages"].append(AIMessage(content="Your itinerary is ready! Moving on to booking assistance."))
    state["status"] = "itinerary_built"
    return state

In [10]:
def booking_assistant(state: PlannerState) -> PlannerState:
    """
    Provide booking recommendations and strategies for the trip.

    Args:
        state (PlannerState): The current workflow state.

    Returns:
        PlannerState: Updated state with booking information.
    """
    preferences = state["travel_preferences"]
    destination = state["selected_destination"]
    itinerary = state["itinerary"]
    system_prompt = """
        You are a travel booking assistant with expertise in reservation strategies.

        Provide **extensively detailed booking recommendations**, including:

        1. **Flight Options**
        - Multiple routes with specific airlines
        - Fare comparison strategies
        - Optimal booking times
        - Layover considerations

        2. **Accommodation Recommendations**
        - Curated listings with platform/direct booking links
        - Property amenities (WiFi, breakfast, accessibility, etc.)
        - Neighborhood advantages and safety

        3. **Activity Reservations**
        - Required bookings (e.g., tours, attractions, events)
        - Advance booking timelines
        - Skip-the-line or VIP access options

        4. **Transportation Bookings**
        - Car rentals (providers, pickup/drop-off info)
        - Train passes or city transport cards
        - Local transit options with pre-booking tips

        5. **Travel Insurance**
        - Plan recommendations by destination/activities
        - Medical, trip cancellation, and baggage coverage
        - Insurance provider comparisons
        - Suggest various insurance options with links to websites

        6. **Baggage Considerations**
        - Airline-specific policies (carry-on vs. checked)
        - Packing strategies and weight tips

        7. **Loyalty Programs & Upgrades**
        - Opportunities to earn/redeem miles or points
        - Upgrade strategies (timing, check-in tips)

        8. **Cancellation Policies**
        - Refund eligibility and cancellation windows
        - Suggested flexible booking options

        9. **Booking Confirmations**
        - Best practices for storing confirmations
        - Centralized digital folders or apps

        10. **Seat Selection Tips**
            - Ideal locations by plane type
            - Avoiding middle seats, maximizing comfort

        11. **Seasonal Pricing Trends**
            - High/low season comparisons
            - Booking windows by destination and event

        12. **Expert Tips**
            - Price tracking tools and alerts
            - Aggregator sites vs. direct booking trade-offs
            - Timeline checklist to ensure timely reservations

        Ensure all recommendations are **optimized for availability, comfort, and cost-effectiveness**.
        """

    response = llm.invoke([
        SystemMessage(content=system_prompt),
        HumanMessage(content=f"Travel preferences: {preferences}, Selected destination: {destination}, Itinerary: {itinerary}")
    ])
    bookings = {
        "recommendations": response.content,
        "timestamp": datetime.now().isoformat()
    }
    state["bookings"] = bookings
    state["messages"].append(AIMessage(content="Here are your booking recommendations. Please provide feedback on the plan."))
    state["status"] = "bookings_provided"
    return state

In [11]:
def trip_exporter(state: PlannerState) -> PlannerState:
    """
    Compile all trip details into a final exportable document.

    Args:
        state (PlannerState): The current workflow state.

    Returns:
        PlannerState: Updated state containing the final trip package.
    """
    preferences = state["travel_preferences"]
    destination = state["selected_destination"]
    budget = state["budget_plan"]
    itinerary = state["itinerary"]
    bookings = state["bookings"]
    system_prompt = """
        You are a travel document preparation specialist with exceptional organizational skills.

        Create a **comprehensive and meticulously structured trip package** that includes:

        1. **Trip Summary**
        - All key logistical elements
        - Contact information for accommodations, transport, and activities
        - Include website links whenever possible

        2. **Chronological Itinerary**
        - Complete day-by-day schedule with precise timings
        - Transportation details and location coordinates (e.g., maps or GPS-ready links)

        3. **Budget Breakdown**
        - Itemized costs by category (e.g., food, lodging, transport, entertainment)
        - Include simple tracking mechanisms (tables, checklists, or trackers)

        4. **Booking & Confirmation Details**
        - Organized by category: flights, hotels, tours, activities
        - Include reference numbers and contact info

        5. **Packing Recommendations**
        - Tailored to destination's climate and activities
        - Include weather-specific gear and region-specific items

        6. **Clothing Suggestions**
        - Outfit suggestions per day, activity, or occasion

        7. **Essential Documents Checklist**
        - Passports, IDs, visas, insurance, tickets, etc.

        8. **Medication Considerations**
        - Prescriptions, travel sickness, vaccination records

        9. **Electronic Device Prep**
        - Adapter types, power banks, connectivity tools

        10. **Cultural Etiquette Guidelines**
            - Dos and don'ts for the destination country or region

        11. **Language Essentials**
            - Key phrases for greetings, transportation, dining, and emergencies

        12. **Safety Information**
            - Travel advisories, personal safety tips

        13. **Emergency Resources**
            - Local emergency numbers
            - Nearest embassies/consulates
            - Closest hospitals or medical centers
            - Provide the website links if possible

        14. **Organization Tips**
            - Digital folder structure (e.g., PDFs, e-tickets)
            - Physical folder layout (e.g., binders, wallets)

        15. **Return Preparation Guidance**
            - Checklist for homecoming (e.g., airport transit, customs, unpacking priorities)

        Deliver this as a complete **travel document system**, formatted for:
        - **Printer-friendly use**
        - **Mobile-optimized access**

        Ensure it is easy to reference throughout the journey.
        """

    response = llm.invoke([
        SystemMessage(content=system_prompt),
        HumanMessage(content=f"""
        Travel preferences: {preferences}
        Selected destination: {destination}
        Budget plan: {budget}
        Itinerary: {itinerary}
        Booking details: {bookings}
        """)
    ])
    final_trip = {
        "package": response.content,
        "created_at": datetime.now().isoformat()
    }
    state["final_trip"] = final_trip
    state["messages"].append(AIMessage(content="Your trip package is ready! Here's everything you need for your journey."))
    state["status"] = "trip_exported"
    return state

# 5. Workflow Construction with LangGraph

- Adds each function as a node to a StateGraph.
- Specifies node execution order (edges).
- Entry point: input_collector.
- Terminal node: trip_exporter (signals workflow completion).
- Compiles the workflow into a graph object.

In [12]:
from langgraph.graph import StateGraph, END

# Create the workflow graph structure
workflow = StateGraph(PlannerState)

# Add all workflow nodes
workflow.add_node("input_collector", input_collector)
workflow.add_node("destination_research", destination_research)
workflow.add_node("collect_selected_destination", collect_selected_destination)
workflow.add_node("budget_planner", budget_planner)
workflow.add_node("itinerary_builder", itinerary_builder)
workflow.add_node("booking_assistant", booking_assistant)
workflow.add_node("trip_exporter", trip_exporter)

# Define workflow logic (edges)
workflow.set_entry_point("input_collector")
workflow.add_edge("input_collector", "destination_research")
workflow.add_edge("destination_research", "collect_selected_destination")
workflow.add_edge("collect_selected_destination", "budget_planner")
workflow.add_edge("budget_planner", "itinerary_builder")
workflow.add_edge("itinerary_builder", "booking_assistant")
workflow.add_edge("booking_assistant", "trip_exporter")
workflow.add_edge("trip_exporter", END)

# Compile workflow for execution
graph = workflow.compile()

# 6. Main Interaction Loop
- process_travel_query(query, planner_state):
    - Handles one user query, advancing the workflow, updating state, and returning assistant responses (or errors).
    - Accepts "exit" to terminate.
    - "continue" advances to next step.
- Sample Interaction:
    - Simulates a conversation, starting with a user’s preferences, continuing through destination selection, and ending with trip export and Markdown file creation.

In [13]:
def process_travel_query(query, planner_state):
    """
    Process a single user query, advancing the workflow and updating state.

    Args:
        query (str): The user's input query.
        planner_state (PlannerState): The current state of the planner.

    Returns:
        tuple: (response (str), updated_state (PlannerState))
    """
    if query.lower() == "exit":
        return "Exiting the travel planner.", planner_state

    # Add new user input to the state, unless just continuing
    if query.lower() != "continue":
        planner_state["messages"].append(HumanMessage(content=query))


    try:
        assistant_response = ""
        for output in graph.stream(planner_state):
            for node, updated_state in output.items():
                print(f"\n[Node: {node}]")
                # Collect new assistant messages
                for message in updated_state["messages"]:
                    if isinstance(message, AIMessage) and message not in planner_state["messages"]:
                        assistant_response += f"{message.content}\n"
                planner_state = updated_state

                # If workflow is complete, export the trip package
                if planner_state["status"] == "trip_exported":
                    final_package = planner_state["final_trip"].get("package", "No package available.")
                    assistant_response += f"\nFinal Trip Package:\n{final_package}\n\nPlanning complete!"
                    return assistant_response, planner_state
        # Prompt user based on workflow state
        if not assistant_response:
            if planner_state["status"] == "initial":
                assistant_response = "Enter your travel preferences (e.g., destination, dates, budget, etc.): "
            elif planner_state["status"] == "awaiting_destination_selection":
                options = planner_state.get("destination_options", {}).get("options", "No options available yet.")
                assistant_response = f"\nSuggested destinations:\n{options}\nPlease select a destination from the suggested options: "
            else:
                assistant_response = "Provide any additional input or feedback (or type 'continue' to proceed): "

        return assistant_response, planner_state

    except Exception as e:
        error_message = (
            f"Error occurred: {e}\nStack trace:\n{traceback.format_exc()}\n"
            "Please check your setup (e.g., API keys, dependencies) and try again."
        )
        return error_message, planner_state

In [14]:
# Initial empty state for the workflow
state = PlannerState(
    travel_preferences={},
    destination_options=[],
    selected_destination={},
    budget_plan={},
    itinerary=[],
    bookings={},
    feedback={},
    final_trip={},
    messages=[],
    status="initial"
)

#simulate several queries
queries = [
    """
I'm planning a family trip to Kenya in June 2025 with my spouse and two children (ages 10 and 14).

- **Budget:** $8,000 (excluding international flights)  
- **Activity Interests:**  
  - Water activities including snorkeling, swimming, and possibly kayaking or sailing along the coast  
  - Wildlife safaris focusing on game drives and guided wildlife viewing in national parks such as Maasai Mara and Amboseli  
- **Accommodation Preferences:** Mid-range hotels or lodges with family rooms or interconnected rooms that offer comfort and kid-friendly amenities  
- **Trip Duration:** 10-12 days  
- **Special Requirements:** No dietary restrictions or mobility concerns; standard health precautions apply

**Request:**  
Please recommend specific regions and itinerary suggestions that provide an ideal balance between coastal water activities and inland wildlife safaris, within the specified budget and accommodation preferences, suitable for a family with children aged 10 and 14.
    """,
    "continue",
    "Nairobi sounds good, let's select it.",
    "exit"
]

print("Welcome to the Travel Planning Assistant!")
current_state = state  # Use the initialized state

for query in queries:
    response, current_state = process_travel_query(query, current_state)
    print(f"\nAssistant: {response}")

    # Save and show response if workflow is complete
    if query.lower() == "exit" or current_state["status"] == "trip_exported":
        file_path = save_file(response, filename="travel_plan", uniqueness_method="datetime")
        clear_output()
        show_md_file(file_path)
        break


Final Trip Package:
**Comprehensive Travel Document System for Kenya Family Trip**

**Trip Summary**

* Destination: Kenya
* Travel Dates: June 2025
* Number of Travelers: 4 (2 adults, 2 children aged 10 and 14)
* Budget: $8,000 (excluding international flights)
* Accommodation Preferences: Mid-range hotels or lodges with family rooms or interconnected rooms that offer comfort and kid-friendly amenities
* Activity Interests: Water activities (snorkeling, swimming, kayaking, sailing), Wildlife safaris (game drives, guided wildlife viewing in national parks)
* Special Requirements: No dietary restrictions, no mobility concerns, standard health precautions apply

**Chronological Itinerary**

* **Day 1: Arrival in Mombasa**
	+ Arrive at Moi International Airport (MBA) and transfer to hotel (approx. 30 minutes)
	+ Spend the day relaxing at the hotel or exploring the nearby Old Town
	+ Dinner at a local restaurant (e.g., Nyama Choma restaurant)
* **Day 2: Mombasa Beach Day**
	+ Spend the day at the beach (e.g., Diani Beach or Tiwi Beach)
	+ Lunch at a beachside restaurant
	+ Sunset cruise along the coast (approx. $50-$100 per person)
* **Day 3: Mombasa to Amboseli National Park**
	+ Fly from Mombasa to Amboseli National Park (approx. 1 hour)
	+ Check-in at a mid-range lodge (e.g., Amboseli Serena Safari Lodge)
	+ Afternoon game drive in the park
	+ Dinner at the lodge
* **Day 4: Amboseli National Park**
	+ Morning and afternoon game drives in the park
	+ Visit the observation deck for a panoramic view of the park
	+ Lunch at the lodge or a picnic in the park
	+ Campfire and stargazing in the evening
* **Day 5: Amboseli to Maasai Mara National Reserve**
	+ Fly from Amboseli to the Maasai Mara National Reserve (approx. 1.5 hours)
	+ Check-in at a mid-range lodge (e.g., Mara Serena Safari Lodge)
	+ Afternoon game drive in the reserve
	+ Dinner at the lodge
* **Day 6: Maasai Mara National Reserve**
	+ Full-day game drive in the reserve
	+ Visit a Maasai village for a cultural experience (approx. $20-$50 per person)
	+ Lunch at the lodge or a picnic in the reserve
	+ Sundowner at the lodge
* **Day 7: Maasai Mara National Reserve**
	+ Morning and afternoon game drives in the reserve
	+ Visit the Mara River for a chance to spot hippos and crocodiles
	+ Lunch at the lodge or a picnic in the reserve
	+ Campfire and stargazing in the evening
* **Day 8: Maasai Mara to Mombasa**
	+ Fly from the Maasai Mara to Mombasa (approx. 1.5 hours)
	+ Spend the day relaxing at the hotel or exploring the nearby Old Town
	+ Dinner at a local restaurant
* **Day 9: Mombasa Beach Day**
	+ Spend the day at the beach (e.g., Diani Beach or Tiwi Beach)
	+ Lunch at a beachside restaurant
	+ Sunset cruise along the coast (approx. $50-$100 per person)
* **Day 10: Mombasa to Nairobi**
	+ Fly from Mombasa to Nairobi (approx. 1 hour)
	+ Spend the day exploring Nairobi (e.g., visit the Giraffe Centre and the National Museum)
	+ Dinner at a local restaurant
* **Day 11: Nairobi**
	+ Spend the day exploring Nairobi (e.g., visit the Elephant Orphanage and the Karen Blixen Museum)
	+ Lunch at a local restaurant
	+ Farewell dinner at a local restaurant
* **Day 12: Departure**
	+ Transfer to the airport for your international departure flight

**Budget Breakdown**

* Transportation: $2,500 - $3,500
* Accommodation: $3,000 - $6,000
* Daily expenses: $1,200 - $2,500
* Entertainment: $500 - $1,000
* Shopping: $800 - $2,000
* Communication: $80 - $200
* Recommended spending money: $4,000 - $8,000
* Extras & contingencies: $1,600 - $3,200
* Total group total: $12,480 - $24,400

**Booking & Confirmation Details**

* Flights:
	+ International flights (4 people): $2,500 - $3,500 (avg. $625 - $875 per person)
	+ Regional connections (domestic flights or buses): $200 - $500 (avg. $50 - $125 per person)
* Accommodations:
	+ Mombasa: Diani Beach Resort or Tijara Beach Resort
	+ Amboseli National Park: Amboseli Serena Safari Lodge or Oltukai Lodge
	+ Maasai Mara National Reserve: Mara Serena Safari Lodge or Mara Intrepids Club
* Activities:
	+ Guided tours and self-guided activities: $50 - $100 per day (avg. $12.50 - $25 per person)
	+ Entrance fees: $10 - $20 per person per activity
* Transportation:
	+ Car rentals (4x4 vehicles): $200 - $500 per day (avg. $50 - $125 per person)
	+ Train passes or city transport cards: $20 - $50 per person

**Packing Recommendations**

* Weather-specific gear:
	+ Lightweight clothing for warm weather
	+ Rain gear (umbrella, raincoat)
	+ Comfortable walking shoes
* Region-specific items:
	+ Insect repellent
	+ Sunscreen
	+ Power adapter for Kenya (Type G)
	+ Portable charger for electronic devices

**Clothing Suggestions**

* Outfit suggestions per day:
	+ Day 1: Casual clothing for arrival and relaxation
	+ Day 2: Beachwear for beach activities
	+ Day 3: Comfortable clothing for game drives and outdoor activities
	+ Day 4: Lightweight clothing for game drives and outdoor activities
	+ Day 5: Comfortable clothing for game drives and outdoor activities
	+ Day 6: Lightweight clothing for game drives and outdoor activities
	+ Day 7: Comfortable clothing for game drives and outdoor activities
	+ Day 8: Casual clothing for relaxation and departure
* Activity-specific clothing:
	+ Snorkeling gear (mask, snorkel, fins)
	+ Swimming goggles
	+ Kayaking or sailing gear (life jacket, paddle)

**Essential Documents Checklist**

* Passports
* IDs
* Visas (if required)
* Travel insurance documents
* Flight itinerary
* Accommodation confirmation
* Activity bookings and confirmations
* Transportation bookings and confirmations
* Health insurance card (if applicable)
* Prescription medication and copies of prescriptions
* Contact information for emergency contacts

**Medication Considerations**

* Prescription medication:
	+ Bring enough medication for the duration of the trip
	+ Pack medication in original packaging
	+ Carry a copy of the prescription
* Travel sickness:
	+ Take motion sickness medication as needed
	+ Avoid heavy meals before traveling
* Vaccination records:
	+ Check with your doctor for recommended vaccinations
	+ Bring copies of vaccination records

**Electronic Device Prep**

* Power adapter for Kenya (Type G)
* Portable charger for electronic devices
* Universal power adapter for charging multiple devices at once
* Waterproof phone case for water activities
* Camera and charger for capturing memories

**Cultural Etiquette Guidelines**

* Respect local customs and traditions:
	+ Dress modestly and remove shoes when entering Maasai villages or churches
	+ Avoid taking photos of people without their permission
	+ Learn a few basic Swahili phrases to communicate with locals
* Dos and don'ts:
	+ Do not touch or handle local animals
	+ Do not litter or pollute the environment
	+ Do not remove plants or rocks from national parks or reserves

**Language Essentials**

* Key phrases for greetings:
	+ Hello: Jambo (HAHM-boh)
	+ Goodbye: Habari (HAH-bah-ree)
* Key phrases for transportation:
	+ Where is...?: Kwa nini? (KWAH nee-nee)
	+ How much?: Ni kiasi gani? (NEE kye-SAH-mee GAH-nee)
* Key phrases for dining:
	+ What is this?: Ni nini? (NEE nee-NEE)
	+ I would like...: Ninataka... (NEE-nah-tah-kah)
* Key phrases for emergencies:
	+ Help!: Msaada! (MEE-sah-DAH)
	+ I need a doctor!: Ninahitaji daktari! (NEE-nah-HEE-tah-jee DAHK-tah-ree)

**Safety Information**

* Travel advisories:
	+ Check the official government website for travel advisories
	+ Register with your country's embassy or consulate
* Personal safety tips:
	+ Be aware of your surroundings and keep valuables secure
	+ Avoid walking alone at night or in isolated areas
	+ Keep emergency contact information handy

**Emergency Resources**

* Local emergency numbers:
	+ Police: 999
	+ Ambulance: 999
	+ Fire department: 999
* Nearest embassies/consulates:
	+ US Embassy: [insert address]
	+ Canadian Embassy: [insert address]
	+ UK Embassy: [insert address]
* Closest hospitals or medical centers:
	+ Mombasa: Coast General Hospital
	+ Nairobi: Kenyatta National Hospital

**Organization Tips**

* Digital folder structure:
	+ Create a folder for each day of the trip
	+ Organize documents and bookings by category (flights, accommodations, activities)
	+ Use labels and tags to categorize and prioritize documents
* Physical folder layout:
	+ Use a binder or folder with separate sections for each day of the trip
	+ Organize documents and bookings by category (flights, accommodations, activities)
	+ Use tabs and dividers to categorize and prioritize documents

**Return Preparation Guidance**

* Checklist for homecoming:
	+ Confirm flight itinerary and booking
	+ Pack essential documents and medications
	+ Notify bank and credit card companies of travel plans
	+ Exchange currency for local currency (Kenyan shillings)

This comprehensive travel document system provides a detailed and organized plan for your Kenya family trip. It includes essential documents, budget breakdown, itinerary, and safety information to ensure a smooth and enjoyable trip.

Planning complete!